# Brain Tumor Classifier

## Import modules

In [1]:
from torchvision.transforms import v2 # Transformations
from model.model_v1 import *
from utils.dataset_loader import *
from utils.loss_tracker import *

## Setup

In [2]:
# Dataset dir path.
DATASET_DIR = "./data"

# Define input the image size (network input).
IMAGE_SIZE = 224

# Define train set and test set transformations.
train_transform = v2.Compose([
    v2.Resize((IMAGE_SIZE, IMAGE_SIZE), antialias=True),                    
    v2.RandomHorizontalFlip(p=0.5),                           
    v2.RandomRotation(degrees=10),                             
    v2.RandomAffine(degrees=0, scale=(0.9, 1.1)),              
    #v2.ToDtype(torch.float32, scale=True),                    
    #v2.Normalize(mean=[0.485, 0.456, 0.406],                  
    #             std=[0.229, 0.224, 0.225]),
    v2.ToTensor(), 
    v2.ToDtype(torch.float32, scale=True)
])

test_transform = v2.Compose([
    v2.Resize((IMAGE_SIZE, IMAGE_SIZE), antialias=True),                    
    v2.ToTensor(), 
    v2.ToDtype(torch.float32, scale=True)
])

# Define the datamodule.
brisc_dm = BRISCDataModule(
    data_dir=DATASET_DIR,
    batch_size=32,
    train_transform=train_transform,
    test_transform=test_transform,
    num_workers=9
)

/usr/lib/python3.13/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


## Define the model

In [3]:
model = ResNetFineTuner(num_classes=4, lr=1e-4, freeze_backbone=True)

/usr/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Train model

In [4]:
# Define the loss tracker callback.
loss_tracker = LossTracker()

# Define the trainer.
trainer = L.Trainer(
    accelerator="auto",
    devices=1,
    max_epochs=3,
    callbacks=[loss_tracker]
)

# Train model.
trainer.fit(model, brisc_dm)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 SUPER') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | ResNet           | 23.5 M | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
8.2 K     Trainable params
23.5 M    Non-trainable params
23.5 M    Total pa

Epoch 0:  48%|█████████████████████████████████████████████████████████████████████████████████████▏                                                                                          | 61/126 [00:01<00:02, 31.10it/s, v_num=5]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined